# Util functions

In [1]:
import matplotlib
# Force matplotlib to not use any Xwindows backend.
matplotlib.use('Agg')

In [4]:
import graphviz as gv
import pydot
import random as python_random
import numpy    
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import coo_matrix
from time import gmtime, strftime
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.lines as mlines
from scipy.sparse import linalg
import numpy as np
import math

In [5]:
def add_nodes(graph, nodes):
    for n in nodes:
        if isinstance(n, tuple):
            graph.node(n[0], **n[1])
        else:
            graph.node(n)
    return graph

def add_edges(graph, edges):
    for e in edges:
        if isinstance(e[0], tuple):
            graph.edge(*e[0], **e[1])
        else:
            graph.edge(*e)
    return graph

In [6]:
def dist(u, w, nodes_pos, A=[], H=[]):
    nodes_pos_u = nodes_pos[u]
    nodes_pos_w = nodes_pos[w]
    length = len(nodes_pos_u)
    dist = 0.0
    for i in range(length):
        dist = dist + (nodes_pos_u[i] - nodes_pos_w[i])**2
        
    return -(dist)**0.5

def node2vec_score(u, w, nodes_pos, A=[], H=[]):
    nodes_pos_u = nodes_pos[u]
    nodes_pos_w = nodes_pos[w]
    return numpy.dot(nodes_pos_u, nodes_pos_w)

def mf_score(u, w, nodes_pos, A, H):
    return numpy.dot(A[int(u)], H.T[int(w)])
    
def svd_score(u, w, nodes_pos, A, H):
    return numpy.dot(A[int(u)], A[int(w)])

In [7]:
def make_dataset(nodes_pos, pos_set, neg_set, functs, A=[], H=[]):
    X = []
    Y = []
    
    for edge in pos_set:
        u, w = edge
        x = []
        try:
            for func in functs:
                x.append(func(u, w, nodes_pos, A, H))
        except:
            continue
        X.append(x)
        Y.append(1)
        
    for edge in neg_set:
        u, w = edge
        x = []
        try:
            for func in functs:
                x.append(func(u, w, nodes_pos, A, H))
        except:
            continue
        X.append(x)
        Y.append(0)
        
    X = numpy.array(X)
    Y = numpy.array(Y)
    return X, Y

In [8]:
def make_sparse_matrix(train_set, n):
    row = []
    col = []
    data = []
    for edge in train_set:
        u = int(edge[0])
        w = int(edge[1])
        row.append(u)
        col.append(w)
        row.append(w)
        col.append(u)
        data.append(1)
        data.append(1)
    return coo_matrix((data, (row, col)), shape=(n, n))

In [9]:
def get_giant_component(nodes, edges):
    edges = set(edges)
    nodes = list(nodes)
    
    node2edges = {node : [] for node in nodes}
    for edge in edges:
        node2edges[edge[0]].append(edge)
        node2edges[edge[1]].append(edge)
    
    def walk(source, visited):
        queue = set()
        if source not in visited:
            queue.add(source)
        while len(queue) > 0:
            current = queue.pop()
            visited.add(current)
            for edge in node2edges[current]:
                if edge[0] == current:
                    added = edge[1]
                else:
                    added = edge[0]
                if added not in visited:
                    queue.add(added)
    
    visited = set()
    walk(python_random.choice(nodes), visited)
    if len(visited) != len(nodes):
        print 'Graph is disconnected, will try to choice giant component'
        while len(visited) < 0.5 * len(nodes):
            visited = set()
            walk(python_random.choice(nodes), visited)
        old_edges_count = len(edges)
        old_nodes_count = len(nodes)
        edges = set([edge for edge in edges if edge[0] in visited and edge[1] in visited])
        nodes = set(visited)
        print 'Giant component: %d of %d nodes, %d of %d edges' % (len(nodes), old_nodes_count, 
                                                                   len(edges), old_edges_count)
    return nodes, edges

In [10]:
def read_train(dataset_name):
    max_id = 0
    file_name = dataset_name + "/train.in"
    fin_train = open(file_name, 'r')
    edges = set()
    nodes = set()
    for line in fin_train:
        line = line.strip()
        # assume undirected!
        u, w = sorted(line.split())
        edges.add((u,w))
        nodes.add(u)
        nodes.add(w)
    fin_train.close()

    nodes, edges = get_giant_component(nodes, edges)
    max_id = max(map(int, nodes))
    return edges, nodes, max_id

In [11]:
# sample edges while keeping graph connected
def safe_sample_edges(nodes, edges, sample_size):   
    edges = set(edges)
    nodes = list(nodes)
    
    edge_label = {}
    node2edges = {node : [] for node in nodes}
    for edge in edges:
        node2edges[edge[0]].append(edge)
        node2edges[edge[1]].append(edge)
        edge_label[edge] = 'keep'
    
    def walk(source, visited):
        queue = set()
        if source not in visited:
            queue.add(source)
        while len(queue) > 0:
            current = queue.pop()
            visited.add(current)
            for edge in node2edges[current]:
                if edge_label[edge] == 'keep':
                    if edge[0] == current:
                        added = edge[1]
                    else:
                        added = edge[0]
                    if added not in visited:
                        queue.add(added)
           
    sampled_edges = set()
    iteration = 0
    while len(sampled_edges) < sample_size:
        candidates = python_random.sample(edges - sampled_edges, sample_size - len(sampled_edges))
        for edge in candidates:
            edge_label[edge] = 'candidate'
        visited = set()
        source = python_random.choice(nodes)
        while len(visited) < len(nodes):
            assert(source not in visited)
            walk(source, visited)
            for edge in candidates:
                if edge_label[edge] == 'candidate':
                    if edge[0] not in visited and edge[1] in visited:
                        edge_label[edge] = 'keep'
                        source = edge[0]
                        break
                    elif edge[1] not in visited and edge[0] in visited:
                        edge_label[edge] = 'keep'
                        source = edge[1]
                        break
                    elif edge[0] in visited and edge[1] in visited:
                        edge_label[edge] = 'remove'
                    else:
                        pass
        for edge in edges:
            if edge_label[edge] == 'remove':
                sampled_edges.add(edge)
            assert(edge_label[edge] != 'candidate')
        print 'Iteration %d, sampled edges %d' % (iteration, len(sampled_edges))
        iteration += 1

    return nodes, edges, sampled_edges

In [13]:
def get_sets(nodes, edges, division=10):
    nodes_size = len(nodes)
    edges_size = len(edges)
    print "Nodes size: " + str(nodes_size)
    print "Edges size: " + str(edges_size)
    test_size = int(edges_size * (division / 100.0))
        
    nodes, edges, pos_edges = safe_sample_edges(nodes, edges, test_size)
    pos_edges = set(pos_edges)
    neg_edges = set()
    while len(neg_edges) < test_size:
        u, w = sorted(map(str, list(python_random.sample(nodes, 2))))
        edge = (u, w)
        if edge not in edges and u != w:
            neg_edges.add(edge)
        
    edges_not_full = edges - pos_edges
    print "Edges not full: " + str(len(edges_not_full))
    
    return pos_edges, neg_edges, edges_not_full

In [14]:
def render_graph(dataset_name, dimension, nodes, edges_not_full):
    graph = gv.Graph(format="dot")
    graph.engine = 'sfdp'
    graph.graph_attr['dim'] = str(dimension)
    graph.graph_attr['dimen'] = str(dimension)
    graph = add_nodes(graph, nodes)
    graph = add_edges(graph, edges_not_full)
    file_name = dataset_name + "/sfdp_graph" #+ str(dimension)
    graph.render(file_name, view=False)
    
    graph.format = "png"
    graph.render(file_name, view=False)
    
    return

In [15]:
def read_edges_dot(dataset_name, dimension):
    file_name = dataset_name + "/sfdp_graph" + ".dot" #+ str(dimension) + ".dot"
    dot_graph = pydot.graph_from_dot_file(file_name)[0]
    dot_nodes = dot_graph.get_nodes()
    nodes_pos = {}
    for node in dot_nodes:
        name = node.get_name()
        if name != 'node' and name != 'graph':
            pos_str = node.get('pos').strip('"')        
            nodes_pos[name] = map(float, pos_str.split(','))
    return nodes_pos

In [16]:
def auc_sfdp(nodes_pos, pos_edges, neg_edges):
    X, Y = make_dataset(nodes_pos, pos_edges, neg_edges, [dist])
    score = roc_auc_score(Y, X)
    print "SFDP " + str(score)
    return score

In [17]:
def auc_nmf(edges, pos_edges, neg_edges, n_components, max_id):
    G = make_sparse_matrix(edges, max_id + 1)
    model = NMF(n_components=n_components, init='random')
    W = model.fit_transform(G)
    H = model.components_
    
    X, Y = make_dataset(None, pos_edges, neg_edges, [mf_score], W, H)
    
    score = roc_auc_score(Y, X)
    print "NMF " + str(score)
    return score

In [18]:
def auc_svd(edges, pos_edges, neg_edges, n_components, max_id):
    G = make_sparse_matrix(edges, max_id + 1)

    U, s, Vh = linalg.svds(G.asfptype(), k=n_components)
    Us = U * s
    
    X, Y = make_dataset(None, pos_edges, neg_edges, [mf_score], Us, Vh)
    
    score = roc_auc_score(Y, X)
    print "SVD " + str(score)
    return score, Us, Vh

In [19]:
import os

def auc_node2vec(dataset_name, edges, pos_edges, neg_edges, dimension):
    node2vec_in = dataset_name + '/' + 'node2vec.in'
    node2vec_out = dataset_name + '/' + 'node2vec.out'

    fin = open(node2vec_in, 'w')
    for edge in edges:
        print>>fin, '%s\t%s' % (edge[0], edge[1])
    fin.close()

    os.system('python ../node2vec/main.py --input "%s" --output "%s" '
              '--dimensions %d --walk-length 80 --p 1 --iter 1' % (node2vec_in, node2vec_out, dimension))
    
    fout = open(node2vec_out, 'r')
    fout.readline() # skip header   
    nodes_pos = {}
    for line in fout:
        fields = line.split()
        nodes_pos[fields[0]] = np.array(map(float, fields[1:]))

    X, Y = make_dataset(nodes_pos, pos_edges, neg_edges, [node2vec_score])
    score = roc_auc_score(Y, X)
    print "node2vec " + str(score)
    return score, nodes_pos

In [20]:
def compute_mean(dim, nodes_pos):
    values = []

    for i in range(dim):
        values.append([])

    for node in nodes_pos:
        for i in range(dim):
            values[i].append(nodes_pos[node][i])
        
    for i in range(dim):
        print np.mean(values[i])
        print np.std(values[i])
        print

In [21]:
from mpl_toolkits.mplot3d import Axes3D

def draw_3d_vectors(dataset_name, vectors, method):
    try:
        plt.rcParams["figure.figsize"] = [15, 15]
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter3D(vectors[:,0], vectors[:,1], vectors[:,2])
        ax.autoscale_view(tight=True)
        file_name = dataset_name + "/3d_%s_as_cloud.png" % method
        fig.savefig(file_name)
    except:
        pass

In [22]:
def draw_graph(dataset_name, nodes_pos, edges):
    vectors = []
    for node in nodes_pos:
        vectors.append(np.array(nodes_pos[node]))
    vectors = np.array(vectors)   
    draw_3d_vectors(dataset_name, vectors, 'sfdp')

In [23]:
def draw_graph_node2vec(dataset_name, nodes_pos):
    vectors = []
    for node in nodes_pos:
        vectors.append(np.array(nodes_pos[node]))
    vectors = np.array(vectors)   
    draw_3d_vectors(dataset_name, vectors, 'node2vec')

In [24]:
def draw_graph_svd(dataset_name, U, V):    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    Vt = np.transpose(V)   
    draw_3d_vectors(dataset_name, np.concatenate((U, Vt)), 'svd')

In [25]:
def run_dim_exp(dataset_name, num_exps, dimensions, division):
    dim_scores_sfdp = {}
    dim_scores_nmf = {}
    dim_scores_svd = {}
    dim_scores_node2vec = {}
    for dimension in dimensions:
            dim_scores_sfdp[dimension] = np.array([])
            dim_scores_nmf[dimension] = np.array([])
            dim_scores_svd[dimension] = np.array([])
            dim_scores_node2vec[dimension] = np.array([])
            
    edges, nodes, max_id = read_train(dataset_name)
    
    for dimension in dimensions:
        print "Dimension " + str(dimension)
        
        for i in range(num_exps):
            print "Launch " + str(i)
            print strftime("%Y-%m-%d %H:%M:%S", gmtime())
            
            pos_edges, neg_edges, edges_not_full = get_sets(nodes, edges, division)

            node2vec_scores, node2vec_nodes_pos = auc_node2vec(dataset_name, edges_not_full, pos_edges, neg_edges, dimension)
            dim_scores_node2vec[dimension] = np.append(dim_scores_node2vec[dimension], node2vec_scores)
            
            if dimension <= 10:
                render_graph(dataset_name, dimension, nodes, edges_not_full)
                nodes_pos = read_edges_dot(dataset_name, dimension)            
                sfdp_scores = auc_sfdp(nodes_pos, pos_edges, neg_edges)
                dim_scores_sfdp[dimension] = np.append(dim_scores_sfdp[dimension], sfdp_scores)
    
            nmf_scores = auc_nmf(edges_not_full, pos_edges, neg_edges, dimension, max_id)
            dim_scores_nmf[dimension] = np.append(dim_scores_nmf[dimension], nmf_scores)
    
            svd_scores, U, V = auc_svd(edges_not_full, pos_edges, neg_edges, dimension, max_id)
            dim_scores_svd[dimension] = np.append(dim_scores_svd[dimension], svd_scores)
               
            if dimension == 3:
                print "Draw graph"
                draw_graph_node2vec(dataset_name, node2vec_nodes_pos)
                draw_graph(dataset_name, nodes_pos, edges)
                draw_graph_svd(dataset_name, U, V)
            
    return dim_scores_sfdp, dim_scores_nmf, dim_scores_svd, dim_scores_node2vec

In [9]:
from numpy import median, absolute, mean
import numpy as np

def mad(data, axis=None):
    return median(absolute(data - median(data, axis)), axis)

def mad_std(x, axis=None):
    return 1.4826 * mad(x, axis)

def robust_mean(x):
    filtered = int(len(x) * 0.2)
    return mean(sorted(x)[filtered:])

In [10]:
print robust_mean(np.array([0.8, 0.9, 0.88, 0.83, 0.84, 0.86, -100]))

0.851666666667


In [28]:
def save_scores(dataset_name, dimensions, ticks,
                dim_scores_sfdp, dim_scores_nmf, dim_scores_svd, dim_scores_node2vec, 
                with_lines, suffix="", show=False, shift=0.15):
    plt.rcParams["figure.figsize"] = [6, 4]
    scores_sfdp = []
    scores_nmf = []
    scores_svd = []
    scores_node2vec = []
    shift = shift
    dimensions_sfdp = []
    dimensions_nmf = []
    dimensions_svd = []
    dimensions_node2vec = []
    err_sfdp = []
    err_nmf = []
    err_svd = []
    err_node2vec = []
    
    for dim in dimensions:
        if dim not in ticks:
            continue
        if dim in dim_scores_sfdp:
            scores_sfdp.append(robust_mean(dim_scores_sfdp[dim]))
        scores_nmf.append(robust_mean(dim_scores_nmf[dim]))
        scores_svd.append(robust_mean(dim_scores_svd[dim]))
        scores_node2vec.append(robust_mean(dim_scores_node2vec[dim]))
        dimensions_sfdp.append(dim - 2 * shift)
        dimensions_nmf.append(dim - shift)
        dimensions_svd.append(dim + shift)
        dimensions_node2vec.append(dim + 2 * shift)
        if dim in dim_scores_sfdp:
            err_sfdp.append(mad_std(dim_scores_sfdp[dim]) * 2)
        err_nmf.append(mad_std(dim_scores_nmf[dim]) * 2)
        err_svd.append(mad_std(dim_scores_svd[dim]) * 2)
        err_node2vec.append(mad_std(dim_scores_node2vec[dim]) * 2)

    print "sfdp scores"
    print scores_sfdp
    print "nmf scores"
    print scores_nmf
    print "svd scores"
    print scores_svd
    print "node2vec scores"
    print scores_node2vec    
    print "sfdp err"
    print err_sfdp
    print "nmf err"
    print err_nmf
    print "svd err"
    print err_svd
    print "node2vec err"
    print err_node2vec
    
    fig, ax = plt.subplots( nrows=1, ncols=1 )
    
    linewidth = 0
    if with_lines:
        linewidth = 1
    
    plt.errorbar(x=dimensions_sfdp, y=scores_sfdp, yerr=err_sfdp, c='g', marker='o', linestyle='-', markersize=2, linewidth=linewidth, elinewidth=1)
    plt.errorbar(x=dimensions_nmf, y=scores_nmf, yerr=err_nmf,c='r', marker='o', linestyle='-.', markersize=2, linewidth=linewidth, elinewidth=1)
    plt.errorbar(x=dimensions_svd, y=scores_svd, yerr=err_svd,c='b', marker='o', linestyle=':', markersize=2, linewidth=linewidth, elinewidth=1)
    plt.errorbar(x=dimensions_node2vec, y=scores_node2vec, yerr=err_node2vec,c='magenta', marker='o', linestyle='--', markersize=2, linewidth=linewidth, elinewidth=1)
    
    plt.xlim([1, max(dimensions) + 1])
    plt.ylim(ymax=1.01)
    plt.xticks(ticks)

    ax.set_xlabel('Dimensionality')
    ax.set_ylabel('AUC')
    
    sfdp = mlines.Line2D([], [], linestyle='-', color='green')
    nmf = mlines.Line2D([], [], linestyle='-.', color='red')
    svd = mlines.Line2D([], [], linestyle=':', color='blue')
    node2vec = mlines.Line2D([], [], linestyle='--', color='magenta')
    ax.legend([sfdp, svd, nmf, node2vec], ["SFDP", "SVD", "NMF", "node2vec"], fontsize = 'small', loc=4)
    fig.suptitle(os.path.basename(dataset_name))
    plt.legend()

    file_name = ""
    dataset_name_lower = os.path.basename(dataset_name).replace(' ', '_').lower()
    if with_lines:
        file_name = dataset_name + "/" + dataset_name_lower + "_dims%s.pdf" % suffix
    else:
        file_name = dataset_name + "/" + dataset_name_lower + "_dims%s_no_lines.pdf" % suffix
    fig.savefig(file_name, format='pdf', bbox_inches='tight')
    if show:
        plt.show()

# Prepare triangulated sphere

In [30]:
# https://github.com/mikedh/trimesh

import trimesh as t
import os

s = t.primitives.Sphere(subdivisions=4)
edges = []
for face in s.faces:
    edges += [
        '\t'.join(map(str,sorted([face[0], face[1]]))),
        '\t'.join(map(str,sorted([face[1], face[2]]))),
        '\t'.join(map(str,sorted([face[0], face[2]])))
    ]

edges = list(set(edges))
print 'Edges count: %d' % len(edges)

if not os.path.exists('../data/3d_sphere_triang'):
    os.makedirs('../data/3d_sphere_triang')
out = open('../data/3d_sphere_triang/train.in', 'w')
for edge in edges:
    print>>out, edge
out.close()

Edges count: 7680


# Launch the experiments

In [31]:
import gc
import sys

dataset_names = ["../data/3d_sphere_triang"]
ns = [5]
fractions = [30]
dimensions = [2,3,4,5,6,7,8,9,10,20,30,40,50,75,100]

for dataset_name, n, fraction in zip(dataset_names, ns, fractions):
    print dataset_name
    dim_scores_sfdp, dim_scores_nmf, dim_scores_svd, dim_scores_node2vec = run_dim_exp(dataset_name, n, dimensions, fraction)
    print "saving"
    print dataset_name
    save_scores(dataset_name, dimensions[:9], dimensions[:9], dim_scores_sfdp, dim_scores_nmf, dim_scores_svd, dim_scores_node2vec, True, '1', False, shift=0.1)
    save_scores(dataset_name, dimensions, [2,10,20,30,40,50,75,100], dim_scores_sfdp, dim_scores_nmf, dim_scores_svd, dim_scores_node2vec, True, '2', False, shift=0.4)    